## Data Collection

### Cài thư viện selenium

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://deve

### Cài đặt các thư viện cần thiết

In [ ]:
import pandas as pd
import os
import requests
import time
import random

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
%pip install -q google-colab-selenium

In [ ]:
import google_colab_selenium as gs

In [ ]:
driver = gs.Chrome()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
WEBDRIVER_DELAY_TIME_INIT = 20 # thời gian chờ
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INIT)

### Thực hiện trích xuất nội dung thơ

In [ ]:
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from tqdm import tqdm

datasets = []
deletion_script = 'arguments[0].parentNode.removeChild(arguments[0]);'

for page_idx in tqdm(range(1, 10)):
    main_url = f'https://www.thivien.net/search-poem.php?PoemType=14&ViewType=1&Country=2&Age[]=2&Page={page_idx}'
    driver.get(main_url)

    #Lấy tất cả các link bài thơ ở trang danh sách trước
    links = []
    titles = []
    try:
        # Chờ danh sách bài thơ hiển thị
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".list-item h4 a")))
        items = driver.find_elements(By.CSS_SELECTOR, ".list-item h4 a")
        for item in items:
            links.append(item.get_attribute('href'))
            titles.append(item.text)
    except TimeoutException:
        print(f"Không tải được trang danh sách tại trang {page_idx}")
        continue

    # Truy cập từng link để lấy nội dung
    for i in range(len(links)):
        poem_url = links[i]
        poem_title = titles[i]

        try:
            driver.get(poem_url)

            # Lấy nội dung bài thơ
            poem_content_tag = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'poem-content')))

            # Xóa các thẻ không cần thiết (i, b) bằng script của bạn
            for tag_name in ['i', 'b']:
                try:
                    unwanted_tags = poem_content_tag.find_elements(By.TAG_NAME, tag_name)
                    for tag in unwanted_tags:
                        driver.execute_script(deletion_script, tag)
                except:
                    pass

            poem_content = poem_content_tag.text

            # Lấy nguồn (source)
            try:
                # Sử dụng selector an toàn hơn
                poem_src_tag = driver.find_element(By.CLASS_NAME, "small")
                poem_src = poem_src_tag.text
            except NoSuchElementException:
                poem_src = ''

            datasets.append({
                'title': poem_title,
                'content': poem_content,
                'source': poem_src,
                'url': poem_url
            })

        except Exception as e:
            print(f"Lỗi tại link: {poem_url} - {e}")

 11%|█         | 1/9 [00:17<02:23, 17.94s/it]

Lỗi tại link: https://www.thivien.net/Nguy%E1%BB%85n-Quang-Di%C3%AAu/Ch%E1%BB%93ng-nh%C3%A0-n%C3%B4ng-khuy%C3%AAn-v%E1%BB%A3/poem-bMgIBQgZSjtMYzV2hb0Xtg - Message: 
Stacktrace:
#0 0x58a499d544ba <unknown>
#1 0x58a4997bd492 <unknown>
#2 0x58a4998129b6 <unknown>
#3 0x58a499812bf1 <unknown>
#4 0x58a49985e4d4 <unknown>
#5 0x58a49985b700 <unknown>
#6 0x58a4998046a2 <unknown>
#7 0x58a499805491 <unknown>
#8 0x58a499d1a479 <unknown>
#9 0x58a499d1d391 <unknown>
#10 0x58a499d06a89 <unknown>
#11 0x58a499d1df4e <unknown>
#12 0x58a499ced070 <unknown>
#13 0x58a499d41ca8 <unknown>
#14 0x58a499d41e78 <unknown>
#15 0x58a499d53828 <unknown>
#16 0x78d6c57d0ac3 <unknown>

Lỗi tại link: https://www.thivien.net/Phan-Ch%C3%A2u-Trinh/D%C3%A1m-ch%E1%BA%BFt-v%C3%AC-ngh%C4%A9a/poem-nn9X6VffhenisKWtM_V7Eg - Message: 
Stacktrace:
#0 0x58a499d544ba <unknown>
#1 0x58a4997bd492 <unknown>
#2 0x58a4998129b6 <unknown>
#3 0x58a499812bf1 <unknown>
#4 0x58a49985e4d4 <unknown>
#5 0x58a49985b700 <unknown>
#6 0x58a4998046a2 <

 22%|██▏       | 2/9 [01:52<07:22, 63.24s/it]

Lỗi tại link: https://www.thivien.net/Khuy%E1%BA%BFt-danh-Vi%E1%BB%87t-Nam/%C4%90%E1%BA%A5t-Qu%E1%BA%A3ng-Nam-ch%C6%B0a-m%C6%B0a-%C4%91%C3%A3-th%E1%BA%A5m/poem-6LJAq5a6jXak90i0Arq1Yw - Message: 
Stacktrace:
#0 0x58a499d544ba <unknown>
#1 0x58a4997bd492 <unknown>
#2 0x58a4998129b6 <unknown>
#3 0x58a499812bf1 <unknown>
#4 0x58a49985e4d4 <unknown>
#5 0x58a49985b700 <unknown>
#6 0x58a4998046a2 <unknown>
#7 0x58a499805491 <unknown>
#8 0x58a499d1a479 <unknown>
#9 0x58a499d1d391 <unknown>
#10 0x58a499d06a89 <unknown>
#11 0x58a499d1df4e <unknown>
#12 0x58a499ced070 <unknown>
#13 0x58a499d41ca8 <unknown>
#14 0x58a499d41e78 <unknown>
#15 0x58a499d53828 <unknown>
#16 0x78d6c57d0ac3 <unknown>



 33%|███▎      | 3/9 [02:13<04:23, 43.99s/it]

...

 78%|███████▊  | 7/9 [04:10<01:07, 33.63s/it]

Lỗi tại link: https://www.thivien.net/Phan-Ch%C3%A2u-Trinh/T%E1%BB%95-ch%E1%BB%A9c-y-t%E1%BA%BF-tinh-t%C6%B0%E1%BB%9Dng/poem-l_CxlkRsbqmMnm5ZyXt_zA - Message: 
Stacktrace:
#0 0x58a499d544ba <unknown>
#1 0x58a4997bd492 <unknown>
#2 0x58a4998129b6 <unknown>
#3 0x58a499812bf1 <unknown>
#4 0x58a49985e4d4 <unknown>
#5 0x58a49985b700 <unknown>
#6 0x58a4998046a2 <unknown>
#7 0x58a499805491 <unknown>
#8 0x58a499d1a479 <unknown>
#9 0x58a499d1d391 <unknown>
#10 0x58a499d06a89 <unknown>
#11 0x58a499d1df4e <unknown>
#12 0x58a499ced070 <unknown>
#13 0x58a499d41ca8 <unknown>
#14 0x58a499d41e78 <unknown>
#15 0x58a499d53828 <unknown>
#16 0x78d6c57d0ac3 <unknown>

Lỗi tại link: https://www.thivien.net/Khuy%E1%BA%BFt-danh-Vi%E1%BB%87t-Nam/Tr%C3%AAn-tr%E1%BB%9Di-c%C3%B3-m%E1%BB%99t-%C3%B4ng-sao/poem-DLfYlIFHIh9EkNsvUEAxAg - Message: 
Stacktrace:
#0 0x58a499d544ba <unknown>
#1 0x58a4997bd492 <unknown>
#2 0x58a4998129b6 <unknown>
#3 0x58a499812bf1 <unknown>
#4 0x58a49985e4d4 <unknown>
#5 0x58a49985b700 <un

 89%|████████▉ | 8/9 [05:29<00:48, 48.23s/it]

Lỗi tại link: https://www.thivien.net/B%C3%B9i-K%E1%BB%B7/Vi%E1%BA%BFng-Thu%C3%BD-Ki%E1%BB%81u/poem-xUhwY0pv-JB9UzA_EEXJZw - Message: 
Stacktrace:
#0 0x58a499d544ba <unknown>
#1 0x58a4997bd492 <unknown>
#2 0x58a4998129b6 <unknown>
#3 0x58a499812bf1 <unknown>
#4 0x58a49985e4d4 <unknown>
#5 0x58a49985b700 <unknown>
#6 0x58a4998046a2 <unknown>
#7 0x58a499805491 <unknown>
#8 0x58a499d1a479 <unknown>
#9 0x58a499d1d391 <unknown>
#10 0x58a499d06a89 <unknown>
#11 0x58a499d1df4e <unknown>
#12 0x58a499ced070 <unknown>
#13 0x58a499d41ca8 <unknown>
#14 0x58a499d41e78 <unknown>
#15 0x58a499d53828 <unknown>
#16 0x78d6c57d0ac3 <unknown>



100%|██████████| 9/9 [06:02<00:00, 40.28s/it]


### Lưu dữ liệu trích xuất thành file csv

In [ ]:
df = pd.DataFrame (datasets)
df.to_csv('poem_dataset.csv', index = True )


In [ ]:
df

,title,content,source,url
0,Á Tế Á ca,"Á Tế Á năm châu là bậc nhất,\nNgười nhiều hơn ...",Bình luận nhanh 1\nMinh Ý Đỗ16/04/2024 09:59\n...,https://www.thivien.net/T%C4%83ng-B%E1%BA%A1t-...
1,Ái quốc,"Nay ta hát một thiên ái quốc,\nYêu gì hơn yêu ...",Bình luận nhanh 2\nHuỳnh Nguyễn Diễm My16/12/2...,https://www.thivien.net/Phan-B%E1%BB%99i-Ch%C3...
2,Bài ca răn cờ bạc,"Nào những đứng anh hùng thục nữ,\nBỏ tài bàn, ...",Bình luận nhanh 0\nBạn đang bình luận với tư c...,https://www.thivien.net/Khuy%E1%BA%BFt-danh-Vi...
3,Bài ký đi chơi núi Ngự Bình,"Năm Đinh Tị tháng ba mùng chín,\nCùng bạn tiên...",Bình luận nhanh 0\nBạn đang bình luận với tư c...,https://www.thivien.net/Ho%C3%A0ng-Y%E1%BA%BFn...
4,Bài nói chuyện với ảnh,"Người đâu cũng giống đa tình\nNgỡ là ai, lại l...",Bình luận nhanh 0\nBạn đang bình luận với tư c...,https://www.thivien.net/T%E1%BA%A3n-%C4%90%C3%...
5,Cải tiến máy móc,"Người ta chí loàn lăng thế giới,\nTrí càng ngà...",Bình luận nhanh 0\nBạn đang bình luận với tư c...,https://www.thivien.net/Phan-Ch%C3%A2u-Trinh/C...
6,Cảm tình anh em lao động đối với cụ cử Lương V...,"Hỡi đồng bào!\nLương chí sĩ nước nhà tạ thế,\n...",Bình luận nhanh 0\nBạn đang bình luận với tư c...,https://www.thivien.net/Khuy%E1%BA%BFt-danh-Vi...
7,Cảnh vui của nhà nghèo,Trong trần thế cảnh nghèo là khổ\nNỗi sinh nha...,Bình luận nhanh 5\nNg Đứcanh06/11/2025 22:08\n...,https://www.thivien.net/T%E1%BA%A3n-%C4%90%C3%...
8,Cần phải học đúng,"Người bẩm sinh tinh anh nung đúc,\nDẫu thông m...",Bình luận nhanh 0\nBạn đang bình luận với tư c...,https://www.thivien.net/Nguy%E1%BB%85n-Phan-L%...
9,Chiêu hồn nước,"Hăm lăm triệu trẻ già trai gái,\nBốn nghìn năm...",Bình luận nhanh 0\nBạn đang bình luận với tư c...,https://www.thivien.net/Ph%E1%BA%A1m-T%E1%BA%A...


In [ ]:
!pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.41.0
!pip install -qq accelerate==0.28.0
!apt install git-lfs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 131.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 17.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2023.10.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 21.4 MB/s eta 0:00:00
Reading pac

In [ ]:
!pip install --upgrade transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.28.0
    Uninstalling accelerate-0.28.0:
      Successfully uninstalled accelerate-0.28.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.0
    Uninstalling transformers-4.41.0:
      Successfully un

In [2]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer , GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset


# Data Preprocessing

In [5]:
DATASET_PATH = '/content/poem_final.csv'
df = pd. read_csv ( DATASET_PATH )
df.head()

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...


In [6]:
def split_content(content):
  samples = []

  poem_parts = content.split('\n\n')
  for poem_part in poem_parts:
    poem_in_lines = poem_part.split('\n')
    if(len(poem_in_lines) == 4):
      samples.append(poem_in_lines)
  return samples
df['content'] = df['content'].apply(lambda x: split_content(x))
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,"[[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhi...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,"[[Chiều vừa xốp trên tay, Chợt nghe thoáng ong...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,"[[Dưới giàn hoa thiên lý, Một mình anh đang ng...","Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","[[Đến, nhiều nơi để đến, Về, trở lại với mình,...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,"[[Đừng bao giờ dại dột, Đem chuyện riêng của m...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"[[Ôi, con sóng chết khô,, vật vờ trong bùn quá...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,"[[Áng dương không biết sầu, Nằm mãi ở trên cao...","Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,[],19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,"[[Không có anh để già, Làm sao em được trẻ, Mu...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [7]:
df_exploded = df.explode('content')
df_exploded.reset_index(drop=True, inplace=True)
df_exploded = df_exploded.dropna(subset = ['content'])
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Cái làm ta hạnh phúc”,"[Rồi thêm chút công việc, Cho ta làm hàng ngày...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Chiều vừa xốp trên tay”,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,1,“Chiều vừa xốp trên tay”,"[Ớt đỏ sao cứ đỏ, Táo chín cho thật vàng, Em đ...","Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
4,2,“Dưới giàn hoa thiên lý...”,"[Dưới giàn hoa thiên lý, Một mình anh đang ngồ...","Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
...,...,...,...,...,...
481,98,Anh biết,"[Đất hiến dâng tuổi trẻ, Trời buông thả tuổi g...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
482,99,Anh bốn mùa lập đông,"[Buồn, thì buồn thiệt đó, Em à, có biết không,...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
483,99,Anh bốn mùa lập đông,"[Vâng, phiến buồn còn đó, Em nào biết phải khô...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
484,99,Anh bốn mùa lập đông,"[Ừ, nỗi buồn vậy đó, Em đừng vội bâng khuâng, ...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


In [9]:
df_exploded ['content'] = df_exploded ['content']. apply (lambda x: '\n'.join (x))
df_exploded

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,0,“Cái làm ta hạnh phúc”,Rồi thêm chút công việc\nCho ta làm hàng ngày\...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
2,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
3,1,“Chiều vừa xốp trên tay”,Ớt đỏ sao cứ đỏ\nTáo chín cho thật vàng\nEm đẹ...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
4,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
...,...,...,...,...,...
481,98,Anh biết,Đất hiến dâng tuổi trẻ\nTrời buông thả tuổi gi...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
482,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
483,99,Anh bốn mùa lập đông,"Vâng, phiến buồn còn đó\nEm nào biết phải khôn...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...
484,99,Anh bốn mùa lập đông,"Ừ, nỗi buồn vậy đó\nEm đừng vội bâng khuâng\nV...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


### Chia tập dữ liệu train/test

In [10]:
TEST_SIZE = 0.1
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset = poem_dataset.train_test_split(test_size = TEST_SIZE)

In [11]:

poem_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 396
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', '__index_level_0__'],
        num_rows: 45
    })
})

In [12]:
MODEL_NAME = 'danghuy1999/gpt2-viwiki'
tokenizer = GPT2Tokenizer.from_pretrained(MODEL_NAME)

tokenizer.pad_token = tokenizer.eos_token
MAX_SEQ_LEN = 100

def preprocess_function (row):
 return tokenizer (row ['content'] ,
 max_length = MAX_SEQ_LEN ,
 padding ='max_length',
 truncation = True)

tokenized_poem_dataset = poem_dataset.map(
   preprocess_function ,
    batched =True ,
    num_proc =4 ,
    remove_columns = poem_dataset ['train'].column_names ,
 )


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

Map (num_proc=4):   0%|          | 0/396 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/45 [00:00<?, ? examples/s]

In [13]:
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer , mlm = False)

# Huấn luyện mô hình

In [14]:
model = GPT2LMHeadModel.from_pretrained (MODEL_NAME )

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/149 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
GPT2LMHeadModel LOAD REPORT from: danghuy1999/gpt2-viwiki
Key                                     | Status     |  | 
----------------------------------------+------------+--+-
transformer.h.{0...11}.attn.masked_bias | UNEXPECTED |  | 
multiple_choice_head.summary.bias       | UNEXPECTED |  | 
transformer.h.{0...11}.attn.bias        | UNEXPECTED |  | 
multiple_choice_head.summary.weight     | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [15]:
training_args = TrainingArguments (
 output_dir ='gpt2_viet_poem_generation',
 save_strategy ='epoch',
 learning_rate =2e-5 ,
 num_train_epochs =10 ,
 weight_decay =0.01 ,
 fp16 = True
 )

In [16]:
trainer = Trainer (
  model =model ,
  args = training_args ,
  train_dataset = tokenized_poem_dataset['train'],
  eval_dataset = tokenized_poem_dataset['test'] ,
  data_collator = data_collator
 )
trainer.train()

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
500,5.555300


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=500, training_loss=5.55530029296875, metrics={'train_runtime': 532.611, 'train_samples_per_second': 7.435, 'train_steps_per_second': 0.939, 'total_flos': 293800421376000.0, 'train_loss': 5.55530029296875, 'epoch': 10.0})

In [18]:
prompt = 'Tiền tài đầy kho\n'
device = 'cuda' if torch.cuda.is_available () else 'cpu'
inputs = tokenizer (prompt , return_tensors ="pt").input_ids .to( device )
outputs = model . generate (inputs ,
max_new_tokens =50 ,
 do_sample =True ,
 top_k =50 ,
 top_p =0.95 ,
 temperature =0.8 ,repetition_penalty =1.2
 )
results = tokenizer . batch_decode ( outputs , skip_special_tokens = True )
results = results [0]
print ()
for line in results . split ('\n') :
 print ( line )

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Tiền tài đầy kho
Giây mở đường đến
Dân số đông rất mạnh
Tiếng núi miền trung không hề!
Chót, buồn đau trước ai?
Khói, nhớ sâu vào tay... người đứng nhanh?
Người bạn khó khăn, người thân đầu
